In [120]:
from scipy.integrate import odeint

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import pickle

### Parameter cell

- Parameters are set within a cell
- Cell is tagged with 'parameters'
- Given parameters are then defined via a dictionary in another notebook

In [121]:
experiment_id = '2'
delta = 0.15
omega = 5.1

### MLFlow tracking

- Tracking is initialized by mlflow.start_run with respective experiment ID
- Metrics/Parameters can be logged via key value relationship
- Artifacts, which are arbitrary files, are given via their local path
- Those are assigned to each run

___________

*Metrics can also be tracked dynamically e.g. step-wise and visualized in the UI*
___________

In [ ]:
with mlflow.start_run(experiment_id=experiment_id) as run:
    
    x0 = 20.0
    v0 = 0.0
    amp = 5.0

    def dgl( U, x ):
        return [ U[1], -2*delta*U[1] - (omega**2)*U[0] + amp*np.cos(2.5*omega*x + 0.5) ]

    xs = np.linspace(0,20,50).tolist()
    y = odeint( dgl, [x0, v0], xs )[:,0]


    for i, (xi,yi) in enumerate(zip(xs, y)):
        
        mlflow.log_metric( 'Displacement', yi, step=i )
        mlflow.log_metric( 'Time', xi, step=i )
    
    sim_data_path = 'artifacts_dir/sim_data/osc_data_{}_{}.npy'.format(omega, delta)
    np.save( sim_data_path, np.array( [xs, y] ) )
    mlflow.log_artifact(sim_data_path)
    
    plot_path = 'artifacts_dir/plots/osc_data_{}_{}.png'.format(omega, delta)
    plt.plot(xs, y)
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)